# BoXHED 2.0 quick start

[BoXHED 2.0](https://arxiv.org/pdf/2103.12591.pdf) is a software package for estimating hazard functions nonparametrically via gradient boosting. It dramatically improves upon BoXHED 1.0 [BoXHED: Boosted eXact Hazard Estimator with Dynamic covariates](http://proceedings.mlr.press/v119/wang20o/wang20o.pdf) in speed. BoXHED 2.0 also allows for more general forms of survival data including recurrent events.

This tutorial demonstrates how to apply BoXHED 2.0 to a synthetic dataset. An expanded user's guide can be found in main.py.

### 1. Importing convenience functions from main.py

**_read_synth** reads the synthetic training data and returns a pandas dataframe.

Input:
* *exp_num*: index for the simulated dataset used in Section 5.3 of the BoXHED 1.0 paper (1, 2, 3, or 4)
* *num_irr*: number of irrelevant covariates used in the simulation (0, 20, or 40)

Output:

A pandas data frame with the following columns:
* *subject*: subject ID
* *t_start*: the start time of an epoch for the patient
* *t_end*: the end time of the epoch
* *X_i*: Values of covariates between *t_start* and *t_end* 

We will see what the input data looks like below

In [1]:
from main import _read_synth

**_read_synth_test** reads the synthetic test data and returns a pandas dataframe. The values of the true hazard function are also provided for accuracy comparisons

Input:
* *exp_num*: index for the simulated dataset used in Section 5.3 of the BoXHED 1.0 paper (1, 2, 3, or 4)
* *num_irr*: number of irrelevant covariates used in the simulation (0, 20, or 40)

Output:

* a pandas data frame with the following columns:
  * *t*: time
  * *X_i*: covariate values at time *t*
* a numpy array containing the values of the true hazard function for each row above


In [2]:
from main import _read_synth_test

### 2. Running an example

In [3]:
exp_num       = 1  
num_irr       = 20  
num_quantiles = 256 #number of candidate split points (locations based on quantiles). Integer from 8 to 256

data = _read_synth(exp_num, num_irr)

In [4]:
data

,subject,t_start,t_end,X_0,X_1,X_2,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,delta
0,1.0,0.010000,0.064333,0.050506,0.193469,0.231674,0.993083,0.851162,0.584319,0.800345,0.723247,0.811566,0.030053,0.889275,0.055881,0.251012,0.344169,0.0
1,1.0,0.248753,0.290695,0.674304,0.153427,0.613957,0.335642,0.273510,0.477107,0.583808,0.122043,0.599093,0.056954,0.443361,0.643540,0.822579,0.658943,0.0
2,1.0,0.397960,0.486245,0.687868,0.694016,0.106414,0.274077,0.884433,0.993644,0.880787,0.169948,0.929970,0.649238,0.573877,0.270658,0.098607,0.523562,0.0
3,1.0,0.486245,0.581648,0.249621,0.922289,0.552174,0.610659,0.336085,0.873602,0.422952,0.551589,0.677242,0.763526,0.323687,0.239204,0.044840,0.859324,0.0
4,1.0,0.619609,0.697989,0.620007,0.757853,0.943695,0.579399,0.457101,0.028805,0.339405,0.905926,0.414710,0.446436,0.151682,0.964303,0.456121,0.844170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151723,10000.0,0.535025,0.605529,0.615905,0.403731,0.070243,0.150162,0.144327,0.468761,0.654210,0.126190,0.397139,0.043732,0.985513,0.243602,0.307620,0.291985,0.0
151724,10000.0,0.605529,0.613142,0.618893,0.032322,0.588856,0.117638,0.915837,0.961482,0.240125,0.247604,0.441394,0.070356,0.944886,0.061320,0.583212,0.907586,0.0
151725,10000.0,0.641568,0.641644,0.180398,0.952714,0.123855,0.848981,0.520086,0.648340,0.398978,0.359882,0.265750,0.545727,0.924673,0.977790,0.850796,0.807079,0.0
151726,10000.0,0.680901,0.764474,0.463660,0.791878,0.854197,0.067081,0.182357,0.598972,0.036312,0.551359,0.858578,0.270542,0.247971,0.232599,0.361775,0.129630,0.0


As can be seen above, $t_{start}<t_{end}$ for each epoch (row). Also, the beginning of one epoch starts no earlier than the end of the previous one, i.e. $t_{{end}_{i}}\leq t_{{start}_{i+1}}$. Delta denotes whether an event (possibly recurrent) occurred at the end of the epoch.

For covariates with missing values, BoXHED 2.0 implements tree splits of the form

Left daughter node: {x<=split.point or x is missing}; Right daughter node: {x>split.point}

or

Left daughter node: {x<=split.point}; Right daughter node: {x>split.point or x is missing}.

Alternatively, the user may choose to impute the missing values, for example by carrying forward the most recent observed value.

Creating an instance of BoXHED to preprocess the training data. boxhed.preprocess() takes 4 arguments:

* *num_quantiles*: the number of candidate split points to try for time and for each covariate. The locations of the split points are based on the quantiles of the training data
* *is_cat*: a list of the column indexes that contain categorical data. The categorical data must be one-hot encoded. For example, if a categorical variable with 3 factors is one hot encoded into binary-valued columns 4,5,6, then is_cat = [4,5,6]
* *weighted*: if set to True, the locations of the candidate split points will be based on weighted quantiles (see Section 3.3 of the BoXHED 2.0 paper)
* *nthreads*: number of CPU threads to use for preprocessing the data

Output:

* *subject*: subject ID for each row in the data frames X, w, and delta
* *X*: each row represents an epoch of the transformed data, and contains the values of the covariates as well as its start time
* *w*: Length of each epoch
* *delta*: Equals one if an event occurred at the end of the epoch; zero otherwise


In [5]:
from boxhed import boxhed
boxhed_ = boxhed(max_depth    = 1,
                 n_estimators = 150)

subjects, X, w, delta = boxhed_.preprocess(
        data             = data,
        num_quantiles    = num_quantiles,
        is_cat           = [],
        weighted         = False,
        nthreads         = 1)

Perform *K*-fold cross-validation to choose the hyperparameters {number of boosted trees, tree depth, learning rate}.

First, specify the candidate values for the hyperparameters to cross-validate on (more trees and/or deeper trees may be needed for other datasets):

In [6]:
param_grid = {'max_depth':    [1, 2, 3, 4, 5],
              'n_estimators': [50, 100, 150, 200, 250, 300],
              'eta':          [0.1, 0.3]}

Next, specify:

* The list of GPU IDs to use for training (we set gpu_list to [-1] to use CPU in this tutorial)
* Training batch size, which is the maximum number of BoXHED2.0 instances trained at any point in time. If we perform 10-fold cross-validation using the above param_grid, we would need to train $5\times 6 \times 10 =300$ instances in total
    * When using GPUs, each GPU will train at most batch_size/len(gpu_list) instances at a time
    * When gpu_list = [-1], batch_size is the number of CPU threads used, with each one training one instance at a time
* The value of *K* in *K*-fold cross-validation

In [7]:
gpu_list = [-1]
batch_size = 6
num_folds = 5

Call the *cv* function to perform *K*-fold cross validation on the training set. This outputs the cross validation results for the different hyperparameter combinations
* *cv_rslts*: mean and st.dev of the log-likelihood value for each hyperparameter combination
* *best_params*: The hyper-parameter combination where the mean log-likelihood value is maximized. *HOWEVER*, we recommend using the one-standard-error rule to select the most parsimonious combination that is within st.dev/$\sqrt{k}$ of the maximum log-likelihood value ($\S$7.10 of Hastie et al. (2009))

Note: Due to a problem the Jupyter notebook has with the multiprocessing package, the code commented out below should be run as a script outside of Jupyter, for example in Spyder. Here we set the hyperparameters to the combination found using 5-fold cross-validation on the data

In [8]:
'''
from model_selection import cv
cv_rslts, best_params = cv(
                          param_grid, 
                          X, 
                          w,
                          delta,
                          subjects, 
                          num_folds,
                          gpu_list,
                          batch_size)
'''
best_params = {'max_depth':    1,
              'n_estimators': 150,
              'eta':          0.1}

Fit BoXHED to the training data:

In [9]:
boxhed_.set_params (**best_params)
boxhed_.fit (X, delta, w)

boxhed(n_estimators=150)

Load the test set and the values of the true hazard function at the test points:

In [10]:
true_haz, test_x = _read_synth_test(exp_num, num_irr) 

Use the fitted model to estimate the value of the hazard function for each row of the test set:

In [11]:
preds = boxhed_.predict(test_x)

Compute the RMSE of the estimates, and its 95% confidence interval:

In [12]:
from main import calc_L2
L2 = calc_L2(preds, true_haz)
L2

[0.17020165844435603, [0.16961137402779425, 0.1707919428609178]]